<a href="https://colab.research.google.com/github/DavidSchneider47/ALIrecommendation/blob/main/data_for_RPA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Install** **Libraries**

In [2]:
!pip install earthengine-api --upgrade
!pip install folium


**Access** **Google** **Earth** **Engine**

In [3]:
import geopandas as gpd
import ee
# Trigger the authentication flow.
ee.Authenticate()
# Initialize the library.
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=qBTWp2JT2Id-zDmEkISSF48TAsSgAZdvO5r-v0KLNBs&tc=Bn_RdcNvK1PbYv4fP9lTUi_-RlHRdUoHFhxX1G3HJ6I&cc=suVgXY42aoDQrP3GYNru5VumKpPJLCMHPBeLiuokdfk

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1Adeu5BUmLPfqIvukzWxYFyL6VByDRX5InfYVEt6ZEQkC8B5gZBgLG5B52oc

Successfully saved authorization token.


**Import** **the** **Transit** **Passenger** **Stations** **File**

In [4]:
from google.colab import files
uploaded = files.upload()

Saving buffered_stations (1).geojson to buffered_stations (1).geojson


In [5]:
gdf = gpd.read_file('buffered_stations (1).geojson')

**Visualize a Passenger Station in the Data Set**

In [8]:
# Specify the Landsat image collection
landsat = ee.ImageCollection('LANDSAT/LC08/C01/T1')

# Assume you want to get an image for the first station in the dataframe
# Create an Earth Engine geometry object from the GeoJSON geometry
aoi = ee.Geometry.Polygon(list(gdf.loc[0, 'geometry'].exterior.coords))

# Filter the Landsat collection to a single image that intersects the polygon
# and was collected closest to a certain date. Adjust the date as needed.
image = landsat.filterBounds(aoi).sort('system:time_start', False).first()

# Define the visualization parameters.
vis_params = {
  'bands': ['B5', 'B4', 'B3'], # These are the RGB bands
  'min': 0,
  'max': 500,
  'gamma': 1.4,
  }


In [18]:
from ipywidgets import interact, widgets
from IPython.display import display as ipy_display  # To avoid conflict with folium's display method
import folium
from folium import Marker, Circle, CircleMarker
import pandas as pd

# Your GeoDataFrame 'gdf' should be defined somewhere here

def plot_map(desired_facility_id):
    if not desired_facility_id:  # Check if the input is empty
        print("Please enter a valid Facility ID.")
        return

    try:
        # Convert the input to integer and filter the GeoDataFrame
        facility_id = int(desired_facility_id)
        filtered_gdf = gdf[gdf['Facility ID'] == facility_id]

        # Proceed only if the Facility ID exists in the data
        if filtered_gdf.empty:
            print("Facility ID not found.")
            return

        # Extract the lat and lon from the filtered GeoDataFrame
        lat = filtered_gdf.iloc[0]['geometry'].centroid.y
        lon = filtered_gdf.iloc[0]['geometry'].centroid.x
    except ValueError:
        print("Invalid input. Please enter a numeric Facility ID.")
        return

    # Set up the map centered around the station
    m = folium.Map(location=[lat, lon], zoom_start=16, control_scale=True)

    # Add a small red dot at the center of the circle
    CircleMarker(location=[lat, lon], radius=3, color='red', fill=True, fill_color='red').add_to(m)

    # Add a circle of radius 1/4 mile (in meters) around the station
    Circle(location=[lat, lon], radius=402.336).add_to(m)

    # Add the satellite image as an overlay (uncomment the following lines if you have the image details)
    # mapid = image.getMapId(vis_params)
    # folium.TileLayer(
    #     tiles=mapid['tile_fetcher'].url_format,
    #     attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    #     overlay=True,
    #     name='median composite',
    # ).add_to(m)

    # Add a layer control panel to the map
    m.add_child(folium.LayerControl())

    # Explicitly display the map
    ipy_display(m)

# Create text box widget for Facility ID input
facility_id_input = widgets.Text(
    value='',
    placeholder='Enter Facility ID',
    description='Facility ID:',
    disabled=False
)

# Display the text box and map based on entered Facility ID
interact(plot_map, desired_facility_id=facility_id_input)


interactive(children=(Text(value='', description='Facility ID:', placeholder='Enter Facility ID'), Output()), …

<function __main__.plot_map(desired_facility_id)>